- RBMT : 언어학자 -> SMT : 언어학자를 해고할수록 성능 증가(n-gram)
- seq2seq는 사실 문제가 있음 -> attention 필수(기억력의 문제 보정)
- 단어 시퀀스에 확률을 할당하기 위한 학습 방법이 이전 단어로부터 다음 단어을 예측하는 것.
- seq2seq로 만든 챗봇은 open domain
- 마지막 단어의 hidden state = context
- beam search : 성능에 영향 큼
- 숙제 : 서브클래싱 구현

## 1. BLEU

## Sentence BLEU score

파이썬 자연어 처리 패키지 NLTK는 기계가 생성한 텍스트 데이터에 대해서 성능을 평가하는 BLEU score를 제공합니다.

In [1]:
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import re
import os
import wget
import urllib
import sentencepiece as sp
import csv
import urllib3
import zipfile
import shutil

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf

In [3]:
ref = [["this", "is", "a", "test"], ["this", "is" "test"]]
cand = ["this", "is", "a", "test"]
score = nltk.translate.bleu_score.sentence_bleu(ref, cand)
print(score)

1.0


## Corpus BLEU score

NLTK는 절이나 문장과 같은 다수의 문장의 묶음에 대해서 BLEU를 측정하는 nltk.translate.bleu_score.corpus_bleu()를 제공합니다. 여기서 refs는 "토큰들의 리스트의 리스트의 리스트" 삼중 리스트여야 합니다. 또한 cand는 "토큰들의 리스트의 리스트"이어야 합니다. 설명만으로는 조금 헷갈리므로 예제를 통해 이해해봅시다.

In [4]:
# two refs for one document
refs = [[["this", "is", "a", "test"], ["this", "is" "test"]]]
cands = [["this", "is", "a", "test"]]
score = nltk.translate.bleu_score.corpus_bleu(refs, cands)
print(score)

1.0


## Cumulative and Individual BLEU score

NTLK의 BLEU는 다른 n-gram들에 대해서 가중치를 달리하여 계산할 수 있도록 해줍니다.  
예를 들어 unigram에만 가중을 100% 주고, 다른 2, 3, 4-gram에 대해서 가중을 주고 싶지않다면 다음과 같이 수행할 수 있습니다.

In [5]:
# 1-gram individual BLEU
ref = [["this", "is", "small", "test"]]
cand = ["this", "is", "a", "test"]
score = nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(1, 0, 0, 0))
print(score)

0.75


C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


위 예제를 각각의 n-gram에 대해서 수행해봅시다.

각 개별적인 n-gram 스코어로부터 각각에게 가중을 주어 가중 기하 평균을 구합니다.  
이를 BLEU-4라고 부릅니다. 이는 nltk.translate.bleu_score.sentence_bleu나 nltk.translate.bleu_score.corpus_bleu의 기본값입니다.

BLEU-1, BLEU-2, BLEU-3, BLEU-4를 각각 구하면 다음과 같습니다.

In [ ]:
# cumulative BLEU scores
ref = [["this", "is", "small", "test"]]
cand = ["this", "is", "a", "test"]
print("Cumulative 1-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(1, 0, 0, 0)))
print("Cumulative 2-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.5, 0.5, 0, 0)))
print("Cumulative 3-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.33, 0.33, 0.33, 0)))
print("Cumulative 4-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.750000
Cumulative 2-gram: 0.500000
Cumulative 3-gram: 0.632878
Cumulative 4-gram: 0.707107


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# BLEU 구현하기

In [ ]:
from collections import Counter
import numpy as np
from nltk import ngrams

## Count : 단순 카운트 함수 구현

In [ ]:
# 단순 카운트 함수
def simple_count(tokens, n): # 토큰화 된 cand 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운트

단순 카운트 구현 함수 테스트

In [ ]:
cand = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = cand.split() #단어 토큰화
result = simple_count(tokens, 1) #토큰화 된 문장, 유니그램의 개수를 구하고자 한다면 n=1
print(result)

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [ ]:
cand = "the the the the the the the"
tokens = cand.split() #단어 토큰화
result = simple_count(tokens, 1)
print(result)

Counter({('the',): 7})


## Count_clip 함수 구현

In [ ]:
def count_clip(cand, ref_list, n):
    cnt_ca = simple_count(cand, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict()

    for ref in ref_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

count_clip 함수는 cand 문장과 ref 문장들, 그리고 카운트 단위가 되는 n-gram에서의 n의 값 이 세 가지를 인자로 입력받아서 countclip을 수행합니다. 여기서는 유니그램 정밀도를 구현하고 있으므로 역시나 n=1로 하여 함수를 실행하면 됩니다.  

또한 count_clip 함수 내부에는 기존에 구현했던 simple_count 함수가 사용된 것을 확인할 수 있습니다. Countclip을 구하기 위해서는 Max_Ref_Count값과 비교하기 위해 Count값이 필요하기 때문입니다. Example 2를 통해 함수가 정상 작동되는지 확인해봅시다.

In [ ]:
cand = "the the the the the the the"
refs = [
    "the cat is on the mat",
    "there is a cat on the mat"
]
result = count_clip(cand.split(),list(map(lambda ref: ref.split(), refs)),1)
print(result)

{('the',): 2}


동일한 예제 문장에 대해서 위의 simple_count 함수는 the가 7개로 카운트되었던 것과는 달리 이번에는 2개로 카운트되었습니다. 이제 위의 두 함수를 사용하여 예제 문장에 대해서 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현해봅시다.

## 보정된 유니그램 정밀도 구현하기

In [ ]:
def modified_precision(cand, ref_list, n):
    clip = count_clip(cand, ref_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(cand, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

In [ ]:
result = modified_precision(cand.split(),list(map(lambda ref: ref.split(), refs)),1) # 유니그램이므로 n = 1
print(result)

0.2857142857142857


소수 값이 나오는데 이는 2/7의 값을 의미합니다. 

이제부터 설명에서 언급하는 "정밀도"는 기본적으로 보정된 정밀도(Modified Precision)라고 가정합니다. 정밀도를 보정하므로서 Ca에서 발생하는 단어 중복에 대한 문제점은 해결되었습니다. 하지만 유니그램 정밀도가 가지는 본질적인 문제점있기에 이제는 유니그램을 넘어 바이그램, 트라이그램 등과 같이 n-gram으로 확장해야 합니다.  

앞서 구현한 함수 simple_count, count_clip, modified_precision은 모두 n-gram의 n을 함수의 인자로 받으므로, n을 1대신 다른 값을 넣어서 실습해보면 바이그램, 트라이그램 등에 대해서도 보정된 정밀도를 구할 수 있습니다.

## 짧은 문장 길이에 대한 패널티(Brevity Penalty)

Ref가 1개라면 Ca와 Ref의 두 문장의 길이만을 가지고 계산하면 되겠지만 여기서는 Ref가 여러 개일 때를 가정하고 있으므로 r은 "모든 Ref들 중에서 Ca와 가장 길이 차이가 작은 Ref의 길이"로 합니다. r을 구하는 코드는 아래와 같습니다.

In [ ]:
def closest_ref_length(cand, ref_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(cand) # ca 길이
    ref_lens = (len(ref) for ref in ref_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)입니다. 또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택합니다. 예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택합니다. closest_ref_length 함수를 통해 r을 구했다면, 이제 BP를 구하는 함수 brevity_penalty를 구현해봅시다.

In [ ]:
def brevity_penalty(cand, ref_list):
    ca_len = len(cand)
    ref_len = closest_ref_length(cand, ref_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # cand가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

## BLEU 함수 구현하기

이제 최종적으로 BLEU 점수를 계산하는 함수 bleu_score를 구현해봅시다.

In [ ]:
def bleu_score(cand, ref_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(cand, ref_list) # 브레버티 패널티, BP

    p_n = [modified_precision(cand, ref_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

위 함수가 동작하기 위해서는 앞서 구현한 simple_count, count_clip, modified_precision, brevity_penalty 4개의 함수 또한 모두 구현되어져 있어야 합니다. 지금까지 구현한 BLEU 코드로 계산된 점수와 NLTK 패키지에 이미 구현되어져 있는 BLEU 코드로 계산된 점수를 비교해봅시다.

## NLTK의 BLEU Vs. 구현한 BLEU 함수

In [ ]:
import nltk.translate.bleu_score as bleu


cand = "It is a guide to action which ensures that the military always obeys the commands of the party"
refs = [
    "It is a guide to action that ensures that the military will forever heed Party commands",
    "It is the guiding principle which guarantees the military forces always being under the command of the Party",
    "It is the practical guide for the army always to heed the directions of the party"
]

# 이번 챕터에서 구현한 코드로 계산한 BLEU 점수
print(bleu_score(cand.split(),list(map(lambda ref: ref.split(), refs))))
# NLTK 패키지 구현되어져 있는 코드로 계산한 BLEU 점수
print(bleu.nltk.translate.bleu_score.sentence_bleu(list(map(lambda ref: ref.split(), refs)),cand.split()))

0.5045666840058485
0.5045666840058485


## 2. Subword Tokenizer

### 2-1. IMDB

In [17]:
# 디렉토리 안에 모든 파일들을 DataFrame 형태로 읽어옵니다.
# 구체적으로 문장(sentence)과 문장의 감정상태의 확신정도(sentiment=1~10)를 읽어옵니다.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# 긍정(postive) 예제와 부정(negative) 예제를 하나의 dataframe으로 합치고 
# 긍정 혹은 부정을 나타내는 polarity 컬럼을 추가하고 데이터를 랜덤하게 섞습니다.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# IMDB 영화 리뷰 데이터셋을 다운받고 전처리를 진행합니다.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(fname="aclImdb.tar.gz", origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", extract=True)

    train_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))
  
    return train_df, test_df

In [20]:
wget.download("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")

100% [........................................................................] 84125825 / 84125825

'aclImdb_v1.tar.gz'

In [32]:
import gzip

gb_file = gzip.open("aclImdb_v1.tar.gz", 'rb')

In [33]:
gb_file

<gzip _io.BufferedReader name='aclImdb_v1.tar.gz' 0x2be045f4648>

In [30]:
zipfile.ZipFile("aclImdb_v1.tar.gz").extractall()

BadZipFile: File is not a zip file

In [35]:
train_df, test_df = download_and_load_datasets()

KeyboardInterrupt: 

In [ ]:
train_df

In [ ]:
train_df["sentence"]

0        This movie was awful. The ending was absolutel...
1        Patrick Channing (Jeff Kober) is a disciple of...
2        The first two-thirds of this biopic of fetish ...
3        Fortunately for us Real McCoy fans (most likel...
4        I wouldn't call it awful, but nothing at all s...
                               ...                        
24995    I loved the Batman tv series and was really lo...
24996    One of the best comedians ever. I've seen this...
24997    One thing I always liked about Robert Ludlum t...
24998    Just Cause takes some of the best parts of thr...
24999    Everything in this film is bad , the story , t...
Name: sentence, Length: 25000, dtype: object

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    train_df["sentence"], target_vocab_size=2**13)

In [1]:
print(tokenizer.subwords)

NameError: name 'tokenizer' is not defined

In [2]:
print(train_df["sentence"][20])

NameError: name 'train_df' is not defined

In [3]:
print("Tokenized sample question: {}".format(tokenizer.encode(train_df["sentence"][20])))

NameError: name 'tokenizer' is not defined

In [ ]:
# train_df에 존재하는 문장 중 일부를 발췌
sample_string = "It"s mind-blowing to me that this film was even made."

# encode
tokenized_string = tokenizer.encode(sample_string)
print ("정수 인코딩 후의 문장 {}".format(tokenized_string))

# encoding한 문장을 다시 decode
original_string = tokenizer.decode(tokenized_string)
print ("기존 문장: {}".format(original_string))

assert original_string == sample_string

정수 인코딩 후의 문장 [135, 7968, 8, 965, 7974, 2405, 34, 7, 105, 13, 14, 32, 18, 78, 677, 7975]
기존 문장: It's mind-blowing to me that this film was even made.


In [ ]:
print("단어 집합의 크기(Vocab size) :", tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8185


In [ ]:
for ts in tokenized_string:
  print ("{} ----> {}".format(ts, tokenizer.decode([ts])))

135 ----> It
7968 ----> '
8 ----> s 
965 ----> mind
7974 ----> -
2405 ----> blow
34 ----> ing 
7 ----> to 
105 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
78 ----> even 
677 ----> made
7975 ----> .


In [ ]:
# 앞서 실습한 문장에 even 뒤에 임의로 xyz 추가
sample_string = "It"s mind-blowing to me that this film was evenxyz made."

# encode
tokenized_string = tokenizer.encode(sample_string)
print ("정수 인코딩 후의 문장 {}".format(tokenized_string))

# encoding한 문장을 다시 decode
original_string = tokenizer.decode(tokenized_string)
print ("기존 문장: {}".format(original_string))

assert original_string == sample_string

정수 인코딩 후의 문장 [135, 7968, 8, 965, 7974, 2405, 34, 7, 105, 13, 14, 32, 18, 6373, 8049, 8050, 990, 677, 7975]
기존 문장: It's mind-blowing to me that this film was evenxyz made.


In [ ]:
for ts in tokenized_string:
  print ("{} ----> {}".format(ts, tokenizer.decode([ts])))

135 ----> It
7968 ----> '
8 ----> s 
965 ----> mind
7974 ----> -
2405 ----> blow
34 ----> ing 
7 ----> to 
105 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
6373 ----> even
8049 ----> x
8050 ----> y
990 ----> z 
677 ----> made
7975 ----> .


### 2-2. 네이버 영화 리뷰

In [ ]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fe80cc83390>)

In [43]:
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

train_data = train_data.dropna(subset=["document"])

In [45]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [47]:
tkn = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_data["document"], target_vocab_size=2**13)

In [83]:
print(f"len(tkn.subwords) : {len(tkn.subwords)}")
print(tkn.subwords[:100])

len(tkn.subwords) : 7997
['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


In [84]:
print(train_data["document"][20])
print(tkn.encode(train_data["document"][20]))
for subw in tkn.encode(train_data["document"][20]):
    print(f"{subw:5d} ----> {tkn.decode([subw])}")

나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님
[669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]
  669 ----> 나름 
 4700 ----> 심오
   17 ----> 한 
 1749 ----> 뜻
    8 ----> 도 
   96 ----> 있는 
  131 ----> 듯
    1 ----> . 
   48 ----> 그냥 
 2239 ----> 학생
    4 ----> 이 
 7466 ----> 선생
   32 ----> 과 
 1274 ----> 놀
 2655 ----> 아나
    7 ----> 는 
   80 ----> 영화는 
  749 ----> 절대 
 1254 ----> 아님


In [89]:
print("이 영화 굉장히 재밌다 킄핫핫ㅎ")
print(tkn.encode("이 영화 굉장히 재밌다 킄핫핫ㅎ"))
for subw in tkn.encode("이 영화 굉장히 재밌다 킄핫핫ㅎ"):
    print(f"{subw:5d} ----> {tkn.decode([subw])}")

이 영화 굉장히 재밌다 킄핫핫ㅎ
[4, 23, 1364, 2157, 8235, 8128, 8130, 8235, 8147, 8169, 8235, 8147, 8169, 393]
    4 ----> 이 
   23 ----> 영화 
 1364 ----> 굉장히 
 2157 ----> 재밌다 
 8235 ----> �
 8128 ----> �
 8130 ----> �
 8235 ----> �
 8147 ----> �
 8169 ----> �
 8235 ----> �
 8147 ----> �
 8169 ----> �
  393 ----> ㅎ


### Sentencepiece (구글의 BPE 구현체)
- Sentencepiece의 학습 데이터로는 빈 칸이 포함되지 않은 문서 집합이어야 합니다.

### IMDB

In [118]:
# wget.download("https://github.com/euphoris/datasets/raw/master/imdb.zip")

100% [..............................................................................] 34005 / 34005

'imdb.zip'

In [123]:
data = pd.read_csv("imdb.zip")

In [124]:
data.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [122]:
with open("review.txt", "w", encoding="utf8") as f:
    f.write("\n".join(df["review"]))

In [129]:
sp.SentencePieceTrainer.Train("--input=review.txt --model_prefix=imdb --vocab_size=1000")

- `input` : 학습시킬 파일
- `model_prefix` : 만들어질 모델 이름
- `vocab_size` : 단어 집합의 크기
- `model_type` : 사용할 모델 (unigram(default), bpe, char, word)
- `max_sentence_length`: 문장의 최대 길이
- `pad_id`, `pad_piece`: pad token id, 값
- `unk_id`, `unk_piece`: unknown token id, 값
- `bos_id`, `bos_piece`: begin of sentence token id, 값
- `eos_id`, `eos_piece`: end of sequence token id, 값
- `user_defined_symbols`: 사용자 정의 토큰

In [137]:
tokens = pd.read_csv("imdb.vocab", sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [139]:
tokens.sample(10)

,0,1
52,an,-5.77179
143,se,-6.69717
130,▁some,-6.59937
508,ial,-8.14767
302,ies,-7.46211
16,a,-4.47403
280,O,-7.35558
299,▁pro,-7.45394
15,e,-4.47045
56,▁you,-5.85125


In [140]:
spp = sp.SentencePieceProcessor()
spp.load("imdb.model")

True

In [144]:
text = df.loc[0, "review"]
print(text)
print(spp.encode_as_pieces(text))

A very, very, very slow-moving, aimless movie about a distressed, drifting young man.
['▁A', '▁very', ',', '▁very', ',', '▁very', '▁slow', '-', 'moving', ',', '▁a', 'im', 'less', '▁movie', '▁about', '▁a', '▁dist', 're', 's', 's', 'ed', ',', '▁dri', 'ft', 'ing', '▁you', 'ng', '▁man', '.']


### 네이버 영화 리뷰

In [145]:
# wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt

In [153]:
ratings_df = pd.read_table("ratings.txt")
ratings_df = ratings_df.dropna(subset=["document"])

In [154]:
ratings_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [156]:
with open("naver_review.txt", "w", encoding="utf8") as f:
    f.write("\n".join(ratings_df["document"]))

입력이 반드시 txt 파일이어야 하므로 txt 파일에 저장해줍니다.

In [157]:
sp.SentencePieceTrainer.Train("--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe")

- `.model`, `.vocab` 파일 두개가 생성 됩니다.  
- `.vocab` 에서 학습된 subwords를 확인할 수 있습니다.

In [158]:
subws = pd.read_csv("naver.vocab", sep="\t", header=None, quoting=csv.QUOTE_NONE)

In [159]:
subws.sample(10)

,0,1
4165,톤,-4162
3403,공,-3400
939,부분,-936
646,▁어디,-643
707,▁천,-704
2688,원이,-2685
3998,엘,-3995
1918,▁좋았어요,-1915
527,스터,-524
181,▁눈,-178


In [164]:
spp = sp.SentencePieceProcessor()
spp.load("naver.model")

True

In [186]:
print(spp.DecodePieces(['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']))
print(spp.IdToPiece(4))
print(spp.PieceToId("영화"))
print(spp.DecodeIds([54, 200, 821, 85]))

뭐 이딴 것도 영화냐.
영화
4
진짜 최고의 영화입니다 ᄏᄏ


enable_sampling=True 일 때 Drop-out이 적용되며 alpha=0.1은 10% 확률로 dropout 한다는 의미이다.

In [190]:
for _ in range(5):
    print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=str, enable_sampling=True, alpha=0.5, nbest_size=-1))

['▁', '진', '짜', '▁최고의', '▁', '영화', '입니다', '▁', 'ᄏ', 'ᄏ']
['▁진짜', '▁', '최', '고', '의', '▁', '영화', '입', '니', '다', '▁', 'ᄏ', 'ᄏ']
['▁', '진짜', '▁', '최고', '의', '▁', '영화', '입', '니', '다', '▁ᄏᄏ']
['▁진', '짜', '▁', '최', '고', '의', '▁영화', '입니다', '▁', 'ᄏ', 'ᄏ']
['▁진짜', '▁', '최', '고', '의', '▁영화입니다', '▁', 'ᄏ', 'ᄏ']


In [189]:
print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=str))
print(spp.encode("진짜 최고의 영화입니다 ㅋㅋ", out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]


## 3. 글자 레벨 기계 번역기 (함수형 API)

In [23]:
url ="http://www.manythings.org/anki/fra-eng.zip"
file = "fra-eng.zip"
cur_dir = os.getcwd()
file_dir = os.path.join(cur_dir, file)

shutil.copyfileobj(urllib3.PoolManager().request("GET", url, preload_content=False), open(file_dir, "wb"))

zipfile.ZipFile(file_dir, "r").extractall(cur_dir)

In [73]:
data = pd.read_csv("fra.txt", names=["src", "tar", "CC"], sep="\t")
data = data[["src", "tar"]]
data = data[:60000] # 6만개만 저장
data["tar"] = data["tar"].apply(lambda x : f"\t {x} \n")
data["src"] = data["src"].str.lower()
data["tar"] = data["tar"].str.lower()

In [93]:
data.head()

,src,tar
0,go.,\t va ! \n
1,hi.,\t salut ! \n
2,hi.,\t salut. \n
3,run!,\t cours ! \n
4,run!,\t courez ! \n


In [91]:
chars_src = set([char for sent in data["src"] for char in sent])
chars_tar = set([char for sent in data["tar"] for char in sent])

n_chars_src = len(chars_src)
n_chars_tar = len(chars_tar)


char2idx_src = {}
char2idx_src["UNK"] = 1
char2idx_src.update({char:idx+2 for idx, char in enumerate(chars_src)})
char2idx_tar = {}
char2idx_tar["UNK"] = 1
char2idx_tar.update({char:idx+2 for idx, char in enumerate(chars_tar)})

In [116]:
enc_data = data["src"].apply(lambda x : [char2idx_src[char] for char in x]).tolist()
dec_data = data["tar"].apply(lambda x : [char2idx_tar[char] for char in x]).tolist()

In [117]:
enc_data = tf.keras.preprocessing.sequence.pad_sequences(enc_data)

array([[ 0,  0,  0, ..., 18, 26, 31],
       [ 0,  0,  0, ..., 25, 34, 31],
       [ 0,  0,  0, ..., 25, 34, 31],
       ...,
       [ 7,  6, 50, ..., 21,  6, 31],
       [ 7,  6, 50, ..., 21,  6, 31],
       [ 7,  6, 50, ..., 13, 48, 31]])

In [109]:
lens = sorted([len(sent) for sent in dec_data])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding="post")
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding="post")

In [ ]:
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, chars_src_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [ ]:
decoder_inputs = Input(shape=(None, chars_tar_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(chars_tar_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
750/750 [==============================] - 13s 17ms/step - loss: 0.7774 - val_loss: 0.6827
Epoch 2/50
750/750 [==============================] - 12s 15ms/step - loss: 0.4753 - val_loss: 0.5522
Epoch 3/50
750/750 [==============================] - 12s 15ms/step - loss: 0.3965 - val_loss: 0.4856
Epoch 4/50
750/750 [==============================] - 12s 15ms/step - loss: 0.3530 - val_loss: 0.4454
Epoch 5/50
750/750 [==============================] - 12s 16ms/step - loss: 0.3241 - val_loss: 0.4198
Epoch 6/50
750/750 [==============================] - 12s 15ms/step - loss: 0.3024 - val_loss: 0.4048
Epoch 7/50
750/750 [==============================] - 12s 16ms/step - loss: 0.2857 - val_loss: 0.3944
Epoch 8/50
750/750 [==============================] - 12s 15ms/step - loss: 0.2723 - val_loss: 0.3816
Epoch 9/50
750/750 [==============================] - 12s 15ms/step - loss: 0.2612 - val_loss: 0.3740
Epoch 10/50
750/750 [==============================] - 12s 15ms/step - loss: 0.251

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, chars_tar_size))
    target_seq[0, 0, tar_to_index["\t"]] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == "\n" or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, chars_tar_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
from nltk.translate.bleu_score import nltk.translate.bleu_score.corpus_bleu

In [4]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print("입력 문장:", lines.src[seq_index])
    print("정답 문장:", lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # "\t"와 "\n"을 빼고 출력
    print("번역기가 번역한 문장:", decoded_sentence[:len(decoded_sentence)-1]) # "\n"을 빼고 출력

NameError: name 'encoder_input' is not defined

In [ ]:
from nltk.translate.bleu_score import nltk.translate.bleu_score.corpus_bleu, SmoothingFunction
smooth_fn = SmoothingFunction()

In [5]:
import numpy as np
actual, predicted = list(), list()

for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    actual.append([lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split()])
    predicted.append(decoded_sentence[:len(decoded_sentence)-1].split())
                  
    print(35 * "-")
    print("입력 문장:", lines.src[seq_index])
    print(lines.src[seq_index].split())
    print("정답 문장:", lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # "\t"와 "\n"을 빼고 출력
    print(lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split())
    print("번역기가 번역한 문장:", decoded_sentence[:len(decoded_sentence)-1]) # "\n"을 빼고 출력
    print(decoded_sentence[:len(decoded_sentence)-1].split())
    
    #print(actual)
    #print(predicted)
    print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
    print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))

NameError: name 'encoder_input' is not defined

In [6]:
import numpy as np
actual, predicted = list(), list()

for seq_index in range(0, len(encoder_input)): # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    actual.append([lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split()])
    predicted.append(decoded_sentence[:len(decoded_sentence)-1].split())

    if seq_index < 10:
      print(35 * "-")
      print("입력 문장:", lines.src[seq_index])
      print(lines.src[seq_index].split())
      print("정답 문장:", lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # "\t"와 "\n"을 빼고 출력
      print(lines.tar[seq_index][1:len(lines.tar[seq_index])-1].split())
      print("번역기가 번역한 문장:", decoded_sentence[:len(decoded_sentence)-1]) # "\n"을 빼고 출력
      print(decoded_sentence[:len(decoded_sentence)-1].split())
    
      print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
      print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))
    
print("BLEU-1: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0), smoothing_function = smooth_fn.method1))
print("BLEU-2: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function = smooth_fn.method1))
print("BLEU-3: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function = smooth_fn.method1))
print("BLEU-4: %f" % nltk.translate.bleu_score.corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function = smooth_fn.method1))

NameError: name 'encoder_input' is not defined

# 단어 레벨 챗봇 (함수형 API)

이 챗봇은 데이터를 매우 적게 학습하였습니다. 더 많은 데이터를 학습할수록 일반화 능력이 높아집니다.

In [ ]:
!wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv

--2020-07-19 00:01:32--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData .csv’

ChatbotData .csv    100%[===================>] 868.99K  --.-KB/s    in 0.09s   

2020-07-19 00:01:32 (9.53 MB/s) - ‘ChatbotData .csv’ saved [889842/889842]



In [ ]:
!ls

'ChatbotData .csv'   sample_data


In [ ]:

# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"":;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv("ChatbotData .csv", encoding="utf-8")
question, answer = list(chatbot_data["Q"]), list(chatbot_data["A"])

In [ ]:

# 데이터 개수
len(question)

11823

In [7]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print("Q : " + question[i])
    print("A : " + answer[i])
    print()

NameError: name 'question' is not defined

In [ ]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [8]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print("Q : " + question[i])
    print("A : " + answer[i])
    print()

NameError: name 'pos_tag' is not defined

In [ ]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [ ]:
# 단어 개수
len(words)

454

In [9]:

# 단어 출력
words[:20]

NameError: name 'words' is not defined

In [ ]:

# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [10]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])

NameError: name 'word_to_index' is not defined

In [11]:
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])

NameError: name 'index_to_word' is not defined

In [ ]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [ ]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]

array([354, 431,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [ ]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]

array([  1, 246, 262,  26, 349,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [ ]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

array([246, 262,  26, 349,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [ ]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:

#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [ ]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ""
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += " "

    return sentence

In [12]:
# 에폭 반복
for epoch in range(20):
    print("Total Epoch :", epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print("accuracy :", history.history["accuracy"][-1])
    print("loss :", history.history["loss"][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1


NameError: name 'model' is not defined

In [ ]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [ ]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input("3박4일 놀러가고 싶다")
input_seq

array([[372, 366, 236, 244, 412, 183,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'여행 은 언제나 좋죠 '

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input("3박4일 같이 놀러가고 싶다")
input_seq

array([[372, 366, 236, 153, 244, 412, 183,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'여행 은 언제나 좋죠 '